In [1]:
import time
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping
import pandas as pd
from prosit_t.wandb_agent.train_utils import get_proteometools_data
import seaborn as sns
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
import os
from dlomix.models import PrositIntensityPredictor

2023-08-29 16:31:41.341266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 16:31:41.507614: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-29 16:31:42.341555: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-29 16:31:42.341620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
data_config = {
    "data_source": {
        "train": "/cmnfs/proj/prosit/Transformer/first_pool_train.parquet",
        "val": "/cmnfs/proj/prosit/Transformer/first_pool_test.parquet",
    },
    "fragmentation": "HCD",
    "batch_size": 1024,
    "seq_length": 30,
}

In [6]:
project_name = "transforming-prosit-first-pool"
run = wandb.init(project=project_name)

wandb: Currently logged in as: mamisashvili-lizi (prosit-compms). Use `wandb login --relogin` to force relogin


In [7]:
model_artifact_name = "prosit-compms/transforming-prosit-first-pool/model-jumping-plasma-61:v44"

In [8]:
model = run.use_artifact(model_artifact_name)

In [9]:
model_dir = model.download()

wandb:   4 of 4 files downloaded.  


In [10]:
model = tf.keras.models.load_model(model_dir, custom_objects={
    "masked_spectral_distance": masked_spectral_distance,
    "masked_pearson_correlation_distance": masked_pearson_correlation_distance
})

2023-08-29 16:38:06.842459: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 16:38:07.210590: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43648 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [11]:
train_data, val_data = get_proteometools_data(data_config)

In [12]:
model.compile(loss=masked_spectral_distance,optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001*0.5))

In [13]:
cb_wandb = WandbCallback()

callback_earlystopping = EarlyStopping(
    monitor="val_loss",
    patience=8,
    min_delta=0.0001,
    restore_best_weights=True,
    verbose=1,
)
callbacks = [cb_wandb, callback_earlystopping]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [ ]:
model.fit(
    train_data,
    validation_data=val_data,
    epochs=200,
    callbacks=callbacks,
)

Epoch 1/200


2023-08-29 16:39:34.206507: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45768966144 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


   1/2534 [..............................] - ETA: 2:52:13 - loss: 0.1466

2023-08-29 16:39:35.381054: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2534/2534 [==============================] - ETA: 0s - loss: 0.1427

2023-08-29 16:46:09.782917: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45768966144 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230829_163804-misid3xa/files/model-best/assets


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230829_163804-misid3xa/files/model-best/assets
wandb: Adding directory to artifact (/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230829_163804-misid3xa/files/model-best)... Done. 0.3s


2534/2534 [==============================] - 444s 174ms/step - loss: 0.1427 - val_loss: 0.1427
Epoch 2/200
1217/2534 [=============>................] - ETA: 3:25 - loss: 0.1425

In [12]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
loss,██▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_loss,██▇▇▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁
best_epoch,89
best_val_loss,0.13517
epoch,90
loss,0.12974
val_loss,0.13527


In [20]:
c = tf.constant([[2]])
d = tf.constant([[1,2,4]])

In [21]:
tf.keras.layers.Multiply()([c,d])

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[2, 4, 8]], dtype=int32)>